### Example: Cholera Pandemic in London
---

#### Source

https://vincentarelbundock.github.io/Rdatasets/doc/HistData/Cholera.html

#### Background

1846-1860 Cholera Pandemic (Wikipedia https://en.wikipedia.org/wiki/1846%E2%80%931860_cholera_pandemic)

In 1855, John Snow discovered that cholera was mainly transmitted through contaminated water supply by examining the cases of cholera in London. Brigham et al. (2004) re-examined Snow's study by using a more advanced statistical method, logistic regression. 

#### List of variables

+ `district` - name of the district in London, a character vector
+ `cholera_drate` - deaths from cholera in 1849 per 10,000 inhabitants, a numeric vector
+ `cholera_deaths` - number of deaths registered from cohlera in 1849, a numeric vector
+ `popn` - population, in the middle of 1849, a numeric vector
+ `elevation` - elevation, in feet above the high water mark, a numeric vector
+ `region` - a grouping of the London districts, a factor with levels West North Central South Kent
+ `water` - water supply region, a factor with levels Battersea New River Kew; see Details
+ `annual_deaths` - annual deaths from all causes, 1838-1844, a numeric vector
+ `pop_dens` - population density (persons per acre), a numeric vector
+ `persons_house` - persons per inhabited house, a numeric vector
+ `house_valpp` - average annual value of house, per person (pounds), a numeric vector
+ `poor_rate` - poor rate precept per pound of howse value, a numeric vector
+ `area` - district area, a numeric vector
+ `houses` - number of houses, a numeric vector
+ `house_val` - total house values, a numeric vector

*Details:*

The supply of water was classified as “Thames, between Battersea and Waterloo Bridges” (central London), “New River, Rivers Lea and Ravensbourne”, and “Thames, at Kew and Hammersmith” (western London). The factor levels use abbreviations for these. The data frame is sorted by increasing elevation above the high water mark.


#### Reference

Bingham P., Verlander, N. Q., and Cheal, M. J. (2004). "John Snow, William Farr and the 1849 outbreak of cholera that affected London: a reworking of the data highlights the importance of the water supply." *Public Health*, 118(6), 387-394, https://doi.org/10.1016/j.puhe.2004.05.007

The following cell loads the cholera data from a CSV file 'Cholera.csv'. Download this file from the keio.jp website and put it and this Jupyter notebook in the same folder.

In [ ]:
import numpy as np
import scipy.stats as st
import scipy.optimize as opt
import pandas as pd
from IPython.display import display
from bokeh.io import show, output_notebook
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, HoverTool, Slider, Span
from bokeh.plotting import figure
output_notebook()

In [ ]:
def beta_hpdi(ci0, alpha, beta, prob):
    def hpdi_conditions(v, a, b, p):
        eq1 = st.beta.cdf(v[1], a, b) - st.beta.cdf(v[0], a, b) - p
        eq2 = st.beta.pdf(v[1], a, b) - st.beta.pdf(v[0], a, b)
        return np.hstack((eq1, eq2))
    return opt.root(hpdi_conditions, ci0, args=(alpha, beta, prob)).x

In [ ]:
def bernoulli_stats(y, n, a0, b0, prob):
    """
        y: the sum of all observations
        n: the number of observations
        a0, b0: the hyperparameters in the beta prior, Beta(a0, b0)
        prob: posterior probability for CI and HPDI
    """
    a = y + a0
    b = n - y + b0
    mean_pi = st.beta.mean(a, b)
    median_pi = st.beta.median(a, b)
    mode_pi = (a - 1.0) / (a + b - 2.0)
    sd_pi = st.beta.std(a, b)
    ci_pi = st.beta.interval(prob, a, b)
    hpdi_pi = beta_hpdi(ci_pi, a, b, prob)
    stats = np.hstack((mean_pi, median_pi, mode_pi, sd_pi, ci_pi, hpdi_pi))
    stats = stats.reshape((1, 8))
    stats_string = ['mean', 'median', 'mode', 'sd', 'ci (lower)', 'ci (upper)', 'hpdi (lower)', 'hpdi (upper)']
    param_string = ['$\\theta$']
    results = pd.DataFrame(stats, index=param_string, columns=stats_string)
    return results, a, b

In [ ]:
cholera = pd.read_csv('Cholera.csv', index_col=0) # Pandas dataframe
display(cholera)

Suppose each death from cholera independently occurs with probability $\theta$. In this context, $\theta$ is interpreted as the true death rate from cholera. We use the uniform prior for $\theta$.

In [ ]:
a0 = 1.0
b0 = 1.0

The parameters in the posterior distribution $\alpha_\star$ and $\beta_\star$ are 

In [ ]:
y = cholera['cholera_deaths'].sum()
n = cholera['popn'].sum()
a_star = y + a0
b_star = n - y + b0

Then we draw the posterior distribution of the death rate.

In [ ]:
q = np.linspace(0.004, 0.008, 1001)
source = ColumnDataSource(
    data=dict(
        q = q,
        prior_pdf = st.beta.pdf(q, a0, b0),
        posterior_pdf = st.beta.pdf(q, a_star, b_star)
    )
)
hover = HoverTool(
    tooltips=[
        ('\u03B8', '@q{0.0000}'), 
        ('prior', '@prior_pdf{0.0000}'),
        ('posterior', '@posterior_pdf{0.0000}')
    ]
)
p = figure(width=400, height=300, 
           tools=[hover], toolbar_location=None, title='Posterior Distribution')
p.line('q', 'posterior_pdf', source=source, line_color='navy', line_width=2,
       legend_label='Posterior distribution')
p.line('q', 'prior_pdf', source=source, line_color='firebrick', line_width=2, line_dash='dashed',
       legend_label='Prior distribution')
p.xaxis.axis_label = '\u03B8'
p.yaxis.axis_label = 'Probability density'
p.legend.location = 'top_left'
p.legend.click_policy = 'hide'
p.legend.border_line_color = p.xgrid.grid_line_color = p.ygrid.grid_line_color = p.outline_line_color = None
show(p)

Using the function `bernoulli_stats()` we alread defined, we compute the posterior statistics on $\theta$.

In [ ]:
prob = 0.95
results = bernoulli_stats(y, n, a0, b0, prob)[0]
display(results)

Next, we compute the posterior statistics of $\theta$ in each district.

In [ ]:
cholera_results = pd.DataFrame( \
    np.vstack([bernoulli_stats(x.cholera_deaths, x.popn, a0, b0, prob)[0].values for x in cholera.itertuples()]),
    columns = ['mean', 'median', 'mode', 'sd', 'ci (lower)', 'ci (upper)', 'hpdi (lower)', 'hpdi (upper)'],
    index = cholera['district'].values)
display(cholera_results)